# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import sys
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [3]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [5]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [6]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 1 face image of the person. It is stored in **"images"** folder with the name User_1. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [19]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

num_images = 1
count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= num_images:
         break
cam.release()
cv2.destroyAllWindows()

# Lets run the face recognizer program :-)

In [20]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from Kunal is 0.9871731
Euclidean distance from Normal User is 0.5516295
Euclidean distance from Root is 0.7043868
Euclidean distance from Sumantra is 0.98603076
Euclidean distance from Kunal is 1.0113516
Euclidean distance from Normal User is 0.5454924
Euclidean distance from Root is 0.7243677
Euclidean distance from Sumantra is 0.9972819
Euclidean distance from Kunal is 1.0983428
Euclidean distance from Normal User is 0.5909114
Euclidean distance from Root is 0.73019195
Euclidean distance from Sumantra is 0.9992552
Euclidean distance from Kunal is 1.2035899
Euclidean distance from Normal User is 0.7026558
Euclidean distance from Root is 0.77170557
Euclidean distance from Sumantra is 1.0133264
Euclidean distance from Kunal is 1.0848299
Euclidean distance from Normal User is 0.7035553
Euclidean distance from Root is 0.7297131
Euclidean distance from Sumantra is 1.0416954
Euclidean distance from Kunal is 1.0109756
Euclidean distance from Normal User is 0.6380882
Eucli

Euclidean distance from Kunal is 1.1438534
Euclidean distance from Normal User is 0.77574724
Euclidean distance from Root is 0.9116373
Euclidean distance from Sumantra is 0.9618696
Euclidean distance from Kunal is 1.0108739
Euclidean distance from Normal User is 0.7741418
Euclidean distance from Root is 0.86367184
Euclidean distance from Sumantra is 0.98704743
Euclidean distance from Kunal is 1.0366563
Euclidean distance from Normal User is 0.76064503
Euclidean distance from Root is 0.8947855
Euclidean distance from Sumantra is 0.9404083
Euclidean distance from Kunal is 1.0809999
Euclidean distance from Normal User is 0.8113655
Euclidean distance from Root is 0.90545285
Euclidean distance from Sumantra is 0.96528494
Euclidean distance from Kunal is 1.0952841
Euclidean distance from Normal User is 0.78144383
Euclidean distance from Root is 0.86743397
Euclidean distance from Sumantra is 0.9554178
Euclidean distance from Kunal is 1.0675504
Euclidean distance from Normal User is 0.75611335

Euclidean distance from Root is 0.725752
Euclidean distance from Sumantra is 1.00165
Euclidean distance from Kunal is 1.0500532
Euclidean distance from Normal User is 0.5953375
Euclidean distance from Root is 0.73587656
Euclidean distance from Sumantra is 0.9876096
Euclidean distance from Kunal is 1.1312381
Euclidean distance from Normal User is 0.5705951
Euclidean distance from Root is 0.6907736
Euclidean distance from Sumantra is 1.0215435
Euclidean distance from Kunal is 1.062352
Euclidean distance from Normal User is 0.6263579
Euclidean distance from Root is 0.69251496
Euclidean distance from Sumantra is 1.0441755
Euclidean distance from Kunal is 1.0465602
Euclidean distance from Normal User is 0.5733103
Euclidean distance from Root is 0.71025956
Euclidean distance from Sumantra is 0.95488685
Euclidean distance from Kunal is 1.0920347
Euclidean distance from Normal User is 0.61610425
Euclidean distance from Root is 0.7762352
Euclidean distance from Sumantra is 0.9364152
Euclidean d

Euclidean distance from Kunal is 1.2173915
Euclidean distance from Normal User is 0.68934125
Euclidean distance from Root is 0.57747394
Euclidean distance from Sumantra is 1.1690547
Euclidean distance from Kunal is 1.1281238
Euclidean distance from Normal User is 0.8109021
Euclidean distance from Root is 0.7262352
Euclidean distance from Sumantra is 1.072693
Euclidean distance from Kunal is 1.1561292
Euclidean distance from Normal User is 0.9967594
Euclidean distance from Root is 0.757173
Euclidean distance from Sumantra is 1.2023202
Euclidean distance from Kunal is 1.2917571
Euclidean distance from Normal User is 0.9373122
Euclidean distance from Root is 0.7492436
Euclidean distance from Sumantra is 1.1926466
Euclidean distance from Kunal is 1.2884085
Euclidean distance from Normal User is 0.99501294
Euclidean distance from Root is 0.77198696
Euclidean distance from Sumantra is 1.2240033
Euclidean distance from Kunal is 1.274791
Euclidean distance from Normal User is 0.84615356
Euclid

Euclidean distance from Kunal is 1.1155558
Euclidean distance from Normal User is 0.8531897
Euclidean distance from Root is 0.8718218
Euclidean distance from Sumantra is 0.8750065
Euclidean distance from Kunal is 1.0526943
Euclidean distance from Normal User is 0.7467449
Euclidean distance from Root is 0.7395943
Euclidean distance from Sumantra is 1.0463573
Euclidean distance from Kunal is 1.190643
Euclidean distance from Normal User is 0.81744474
Euclidean distance from Root is 0.74089956
Euclidean distance from Sumantra is 1.0382208
Euclidean distance from Kunal is 1.1529758
Euclidean distance from Normal User is 0.6341358
Euclidean distance from Root is 0.75151426
Euclidean distance from Sumantra is 0.9280839
Euclidean distance from Kunal is 1.2655096
Euclidean distance from Normal User is 0.65107036
Euclidean distance from Root is 0.811704
Euclidean distance from Sumantra is 0.89609945
Euclidean distance from Kunal is 1.2019447
Euclidean distance from Normal User is 0.6048449
Eucli

...

Euclidean distance from Kunal is 1.1515272
Euclidean distance from Normal User is 0.77854824
Euclidean distance from Root is 0.90908355
Euclidean distance from Sumantra is 0.8233447
Euclidean distance from Kunal is 1.1606606
Euclidean distance from Normal User is 0.63111717
Euclidean distance from Root is 0.7807057
Euclidean distance from Sumantra is 0.8698528
Euclidean distance from Kunal is 1.0900828
Euclidean distance from Normal User is 0.6243033
Euclidean distance from Root is 0.75336415
Euclidean distance from Sumantra is 0.9629164
Euclidean distance from Kunal is 1.1739318
Euclidean distance from Normal User is 0.63943696
Euclidean distance from Root is 0.8066857
Euclidean distance from Sumantra is 0.97308034
Euclidean distance from Kunal is 1.2669412
Euclidean distance from Normal User is 0.7660312
Euclidean distance from Root is 0.7332787
Euclidean distance from Sumantra is 1.1698215
Euclidean distance from Kunal is 1.1015358
Euclidean distance from Normal User is 0.7012885
Eu

Euclidean distance from Kunal is 1.143163
Euclidean distance from Normal User is 0.47511193
Euclidean distance from Root is 0.68034506
Euclidean distance from Sumantra is 0.91610104
Euclidean distance from Kunal is 1.1446176
Euclidean distance from Normal User is 0.5853004
Euclidean distance from Root is 0.8248799
Euclidean distance from Sumantra is 0.95025885
Euclidean distance from Kunal is 1.1039538
Euclidean distance from Normal User is 0.6134068
Euclidean distance from Root is 0.81065065
Euclidean distance from Sumantra is 0.980611
Euclidean distance from Kunal is 1.2385154
Euclidean distance from Normal User is 0.55889183
Euclidean distance from Root is 0.7014726
Euclidean distance from Sumantra is 0.94334596
Euclidean distance from Kunal is 1.1903036
Euclidean distance from Normal User is 0.51624286
Euclidean distance from Root is 0.7268055
Euclidean distance from Sumantra is 0.9126955
Euclidean distance from Kunal is 1.172412
Euclidean distance from Normal User is 0.54819494
Eu

Euclidean distance from Kunal is 1.1258882
Euclidean distance from Normal User is 0.6281802
Euclidean distance from Root is 0.6923655
Euclidean distance from Sumantra is 0.9670176
Euclidean distance from Kunal is 1.0691825
Euclidean distance from Normal User is 0.61433566
Euclidean distance from Root is 0.7476029
Euclidean distance from Sumantra is 0.91587126
Euclidean distance from Kunal is 1.1211009
Euclidean distance from Normal User is 0.6102966
Euclidean distance from Root is 0.6533038
Euclidean distance from Sumantra is 0.97540635
Euclidean distance from Kunal is 1.1376292
Euclidean distance from Normal User is 0.63827276
Euclidean distance from Root is 0.77254874
Euclidean distance from Sumantra is 0.88570577
Euclidean distance from Kunal is 1.1187755
Euclidean distance from Normal User is 0.6556716
Euclidean distance from Root is 0.7758149
Euclidean distance from Sumantra is 0.93007183
Euclidean distance from Kunal is 1.1399835
Euclidean distance from Normal User is 0.59780234


Euclidean distance from Kunal is 1.0964676
Euclidean distance from Normal User is 0.6778377
Euclidean distance from Root is 0.77794576
Euclidean distance from Sumantra is 0.9132685
Euclidean distance from Kunal is 1.063777
Euclidean distance from Normal User is 0.7033536
Euclidean distance from Root is 0.78115106
Euclidean distance from Sumantra is 0.9343799
Euclidean distance from Kunal is 1.0042146
Euclidean distance from Normal User is 0.65052634
Euclidean distance from Root is 0.721538
Euclidean distance from Sumantra is 0.9876146
Euclidean distance from Kunal is 1.0750859
Euclidean distance from Normal User is 0.6685664
Euclidean distance from Root is 0.81304276
Euclidean distance from Sumantra is 0.8533077
Euclidean distance from Kunal is 1.114205
Euclidean distance from Normal User is 0.78331333
Euclidean distance from Root is 0.8936388
Euclidean distance from Sumantra is 0.78196335
Euclidean distance from Kunal is 1.1650134
Euclidean distance from Normal User is 0.6240294
Eucli

Euclidean distance from Kunal is 1.1460971
Euclidean distance from Normal User is 0.6978857
Euclidean distance from Root is 0.8068108
Euclidean distance from Sumantra is 0.8503121
Euclidean distance from Kunal is 1.0965785
Euclidean distance from Normal User is 0.61511993
Euclidean distance from Root is 0.80221313
Euclidean distance from Sumantra is 0.86050713
Euclidean distance from Kunal is 1.1073759
Euclidean distance from Normal User is 0.5768942
Euclidean distance from Root is 0.7516457
Euclidean distance from Sumantra is 0.9114356
Euclidean distance from Kunal is 1.1089474
Euclidean distance from Normal User is 0.65117794
Euclidean distance from Root is 0.7883224
Euclidean distance from Sumantra is 0.82175153
Euclidean distance from Kunal is 1.1937692
Euclidean distance from Normal User is 0.64132077
Euclidean distance from Root is 0.794484
Euclidean distance from Sumantra is 0.86250687
Euclidean distance from Kunal is 1.1015438
Euclidean distance from Normal User is 0.62548286
E